<a href="https://colab.research.google.com/github/leejc0404/2-/blob/main/sso_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask-socketio eventlet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.1/364.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.2 MB/s eta 0:00:00


In [5]:
from flask import Flask, render_template_string, request, jsonify
from datetime import datetime
import json
import threading
import time

In [6]:
app = Flask(__name__)

In [13]:
from pyngrok import ngrok
ngrok.set_auth_token("여기에_당신의_authtoken_입력")

In [16]:
# ========================================
# 🛠️ 오류 해결된 안정적인 Colab Flask 앱
# ========================================

# 1️⃣ 필수 라이브러리 설치 (경고 숨김)
import warnings
warnings.filterwarnings('ignore')

!pip -q install flask
print("✅ Flask 설치 완료")

# 2️⃣ 라이브러리 import 및 설정
from flask import Flask
import threading
import time
import os
import sys
from contextlib import redirect_stderr
from io import StringIO

# 3️⃣ HTML 파일 생성 (이모지 안전 처리)
html_content = """<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>타임어택 퀴즈 이벤트</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Malgun Gothic', -apple-system, BlinkMacSystemFont, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
            color: white;
        }

        .container {
            max-width: 500px;
            margin: 0 auto;
            background: rgba(255, 255, 255, 0.1);
            backdrop-filter: blur(10px);
            border-radius: 20px;
            padding: 30px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
            border: 1px solid rgba(255, 255, 255, 0.2);
        }

        .header {
            text-align: center;
            margin-bottom: 30px;
        }

        .title {
            font-size: 2em;
            margin-bottom: 10px;
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
            background: linear-gradient(45deg, #FFD700, #FFA500);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
        }

        .timer {
            font-size: 1.2em;
            color: #FFD700;
            margin-bottom: 20px;
            background: rgba(255, 215, 0, 0.2);
            padding: 10px;
            border-radius: 10px;
            border: 2px solid #FFD700;
        }

        .current-time {
            font-size: 1em;
            color: #FFE4B5;
            margin-bottom: 10px;
        }

        .question-container {
            background: rgba(255, 255, 255, 0.2);
            border-radius: 15px;
            padding: 25px;
            margin-bottom: 25px;
            border: 1px solid rgba(255, 255, 255, 0.3);
        }

        .question-time {
            font-size: 1.3em;
            color: #FFD700;
            font-weight: bold;
            margin-bottom: 15px;
            text-align: center;
        }

        .question {
            font-size: 1.1em;
            line-height: 1.6;
            margin-bottom: 20px;
            white-space: pre-line;
            background: rgba(0, 0, 0, 0.2);
            padding: 15px;
            border-radius: 10px;
            border-left: 4px solid #FFD700;
        }

        .input-group {
            display: flex;
            gap: 10px;
            margin-bottom: 20px;
        }

        .answer-input {
            flex: 1;
            padding: 15px;
            border: none;
            border-radius: 10px;
            font-size: 1.1em;
            background: rgba(255, 255, 255, 0.9);
            color: #333;
        }

        .submit-btn {
            padding: 15px 25px;
            background: linear-gradient(45deg, #FF6B6B, #FF8E53);
            border: none;
            border-radius: 10px;
            color: white;
            font-size: 1em;
            font-weight: bold;
            cursor: pointer;
        }

        .result {
            padding: 20px;
            border-radius: 10px;
            margin-top: 20px;
            text-align: center;
            font-size: 1.2em;
            font-weight: bold;
        }

        .correct {
            background: linear-gradient(45deg, #4CAF50, #45a049);
            border: 2px solid #4CAF50;
        }

        .incorrect {
            background: linear-gradient(45deg, #f44336, #da190b);
            border: 2px solid #f44336;
        }

        @media (max-width: 480px) {
            .container {
                margin: 10px;
                padding: 20px;
            }

            .title {
                font-size: 1.5em;
            }

            .input-group {
                flex-direction: column;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1 class="title">🎮 타임어택 퀴즈 이벤트</h1>
            <div class="current-time" id="current-time"></div>
            <div class="timer" id="timer">로딩 중...</div>
        </div>

        <div id="game-area">
            <div class="question-container">
                <div class="question-time" id="question-time"></div>
                <div class="question" id="question">문제를 불러오는 중...</div>
                <div class="input-group">
                    <input type="text" class="answer-input" id="answer" placeholder="정답을 입력하세요">
                    <button class="submit-btn" onclick="submitAnswer()">제출</button>
                </div>
            </div>
        </div>

        <div id="result"></div>
    </div>

    <script>
        const QUESTIONS = {
            14: {
                hint: `힌트 : 소설 『수난이대』(하근찬)

이 소설에서 O은 상징적으로 반복적으로 등장합니다. 주인공 박만도는 한쪽 O을 잃고, 남은 OO으로 아들을 감싸 안으려는 장면이 인상적입니다.

여기서 OO은 삶의 고통과 끈질긴 생명력, 부성애를 상징합니다. "O로 내 목을 감아야 될 끼다."라는 대사처럼, OO은 단순한 신체 부위를 넘어 감정과 의지의 매개체로 그려집니다.

여기서 OO에 들어갈 말은!?`,
                answer: "팔뚝",
                prize: "이쁘니 상품권"
            },
            16: {
                hint: "여태 OO을 소혜보다 좋아하는 사람을 본적이 없다!! 여기서 OO에 들어갈 말은?",
                answer: "과일",
                prize: "달콤한 과일 세트"
            },
            18: {
                hint: "갈갈갈... (대신 [브랜드][용도][명칭] 다 맞춰야 함. 8글자)",
                answer: "닌자휴대용블랜더",
                prize: "건강한 스무디 쿠폰"
            },
            20: {
                hint: '"강아지꺼 뺏은거 같아", 4월 25일, [브랜드][명칭], 6글자',
                answer: "잔스포츠가방",
                prize: "스타일리시한 가방"
            },
            22: {
                hint: '"내일 필요한 거지... 활활 타라~", 5글자',
                answer: "마시멜로우",
                prize: "달콤한 캠핑 세트"
            }
        };

        let currentQuestion = null;

        function getCurrentQuestion() {
            const now = new Date();
            const currentHour = now.getHours();

            const availableHours = Object.keys(QUESTIONS).map(h => parseInt(h)).sort((a, b) => b - a);

            for (let hour of availableHours) {
                if (currentHour >= hour) {
                    return { data: QUESTIONS[hour], hour: hour };
                }
            }

            return { data: QUESTIONS[14], hour: 14 };
        }

        function getTimeUntilNext() {
            const now = new Date();
            const currentHour = now.getHours();

            const nextHours = Object.keys(QUESTIONS).map(h => parseInt(h)).filter(h => h > currentHour);

            if (nextHours.length > 0) {
                const nextHour = Math.min(...nextHours);
                const nextTime = new Date();
                nextTime.setHours(nextHour, 0, 0, 0);

                const timeDiff = nextTime - now;
                return Math.floor(timeDiff / 1000);
            }

            return 0;
        }

        function updateCurrentTime() {
            const now = new Date();
            const timeString = now.toLocaleString('ko-KR', {
                year: 'numeric',
                month: 'long',
                day: 'numeric',
                hour: '2-digit',
                minute: '2-digit',
                second: '2-digit'
            });
            document.getElementById('current-time').textContent = `현재 시간: ${timeString}`;
        }

        function updateTimer() {
            const seconds = getTimeUntilNext();
            const timerElement = document.getElementById('timer');

            if (seconds <= 0) {
                timerElement.textContent = '🔥 현재 진행 중!';
                return;
            }

            const hours = Math.floor(seconds / 3600);
            const minutes = Math.floor((seconds % 3600) / 60);
            const secs = seconds % 60;

            if (hours > 0) {
                timerElement.textContent = `⏰ 다음 문제까지: ${hours}시간 ${minutes}분 ${secs}초`;
            } else {
                timerElement.textContent = `⏰ 다음 문제까지: ${minutes}분 ${secs}초`;
            }
        }

        function updateQuestion() {
            const questionInfo = getCurrentQuestion();
            currentQuestion = questionInfo;

            const timeNames = {
                14: '오후 2시',
                16: '오후 4시',
                18: '오후 6시',
                20: '오후 8시',
                22: '오후 10시'
            };

            document.getElementById('question-time').textContent = `🕐 ${timeNames[questionInfo.hour]} 문제`;
            document.getElementById('question').textContent = questionInfo.data.hint;
            document.getElementById('result').innerHTML = '';
            document.getElementById('answer').value = '';
        }

        function submitAnswer() {
            const userAnswer = document.getElementById('answer').value.trim();
            if (!userAnswer) return;

            const correctAnswer = currentQuestion.data.answer.replace(/\\s/g, '');
            const userAnswerClean = userAnswer.replace(/\\s/g, '');

            const isCorrect = userAnswerClean.toLowerCase() === correctAnswer.toLowerCase();
            const resultDiv = document.getElementById('result');

            if (isCorrect) {
                resultDiv.innerHTML = `
                    <div class="result correct">
                        <div>🎉 정답! 축하합니다! 🎉</div>
                        <div>🎁 상품: ${currentQuestion.data.prize}</div>
                        <div style="margin-top: 10px; font-size: 0.9em;">
                            정답은 "${currentQuestion.data.answer}" 였습니다!
                        </div>
                    </div>
                `;
            } else {
                resultDiv.innerHTML = `
                    <div class="result incorrect">
                        <div>😅 틀렸징~ 오답이징~</div>
                        <div>다시 도전해보세요!</div>
                        <div style="margin-top: 10px; font-size: 0.9em;">
                            힌트: 답은 ${currentQuestion.data.answer.length}글자입니다
                        </div>
                    </div>
                `;
            }

            setTimeout(() => {
                document.getElementById('answer').focus();
            }, 1000);
        }

        document.getElementById('answer').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                submitAnswer();
            }
        });

        function init() {
            updateCurrentTime();
            updateQuestion();
            updateTimer();

            setInterval(() => {
                updateCurrentTime();
                updateTimer();
            }, 1000);

            setInterval(() => {
                const newQuestion = getCurrentQuestion();
                if (newQuestion.hour !== currentQuestion.hour) {
                    updateQuestion();
                }
            }, 60000);
        }

        window.onload = init;
    </script>
</body>
</html>"""

# HTML 파일 저장
with open('index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("✅ HTML 파일 생성 완료")

# 4️⃣ Flask 앱 설정 (오류 처리 강화)
class SafeFlask(Flask):
    """오류를 안전하게 처리하는 Flask 클래스"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 디버그 모드 비활성화로 _debugpy_repr 오류 방지
        self.config['DEBUG'] = False
        # 로깅 레벨 조정
        import logging
        logging.getLogger('werkzeug').setLevel(logging.ERROR)

# Flask 앱 생성
app = SafeFlask(__name__, static_folder='.', static_url_path='')

@app.route('/')
def home():
    """메인 페이지 반환"""
    try:
        with open('index.html', 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"<h1>페이지 로드 오류</h1><p>{str(e)}</p>"

# 5️⃣ 안전한 서버 실행 함수
def run_flask_safely():
    """Flask 서버를 안전하게 실행"""
    try:
        # 경고 메시지 숨김
        with redirect_stderr(StringIO()):
            app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False, threaded=True)
    except Exception as e:
        print(f"⚠️  서버 실행 중 오류 발생: {e}")

# 6️⃣ 백그라운드에서 Flask 실행
print("🚀 Flask 서버를 시작합니다...")
flask_thread = threading.Thread(target=run_flask_safely, daemon=True)
flask_thread.start()

# 서버 시작 대기
time.sleep(3)

# 7️⃣ Colab 내장 기능으로 안전하게 표시
try:
    from google.colab import output
    print("✅ 서버가 성공적으로 시작되었습니다!")
    print("📱 아래에서 퀴즈 앱을 확인하세요:")

    # iframe으로 안전하게 표시
    output.serve_kernel_port_as_iframe(5000, height=800)

except Exception as e:
    print(f"⚠️  Colab 연동 중 오류: {e}")
    print("🌐 직접 접속하려면 다른 브라우저 탭에서 localhost:5000을 열어보세요")

print("\n🎯 앱이 실행 중입니다. 문제는 시간에 따라 자동으로 업데이트됩니다!")
print("   - 오후 2시: 팔뚝 문제")
print("   - 오후 4시: 과일 문제")
print("   - 오후 6시: 닌자휴대용블랜더 문제")
print("   - 오후 8시: 잔스포츠가방 문제")
print("   - 오후 10시: 마시멜로우 문제")

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

✅ Flask 설치 완료
✅ HTML 파일 생성 완료
🚀 Flask 서버를 시작합니다...
 * Serving Flask app '__main__'
 * Debug mode: off


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

✅ 서버가 성공적으로 시작되었습니다!
📱 아래에서 퀴즈 앱을 확인하세요:


<IPython.core.display.Javascript object>


🎯 앱이 실행 중입니다. 문제는 시간에 따라 자동으로 업데이트됩니다!
   - 오후 2시: 팔뚝 문제
   - 오후 4시: 과일 문제
   - 오후 6시: 닌자휴대용블랜더 문제
   - 오후 8시: 잔스포츠가방 문제
   - 오후 10시: 마시멜로우 문제
